# **Introducción al análisis de datos en Python** 
#### Profesor: Lucas Gómez Tobón

## Clase 5. Pandas Avanzado

### Unir bases de datos (`merge`)

En la sesión anterior aprendimos como concatenar filas y columnas de diferentes bases de datos. Para hacer esto es necesario que la cantidad de columnas y filas, respectivamente, de los dataframes a juntar sean los mismos y que sus índices o llaves también lo sean.

No obstante, muchas veces cuando trate de juntar bases de datos, notará que no necesariamente todas las llaves están presentes en ambas bases de datos, o que incluso, a cada fila de la base izquierda, querrá pegarle más de una fila de la base derecha, o viceversa.

A la hora de hacer pegues más complejos, hablamos de que vamos a utilizar un `merge`. 

Comencemos con la sintaxis del `merge`. Para pegar dos bases de datos, usted usará un comando similar al siguiente:

```python
pd.merge(left = left_dataframe, right = right_dataframe, on = "alguna(s)_columa(s)", how = "left|right|inner|outer")`
```

Los argumentos que toma la función son:
- `left`: dataframe que va de primero.
- `right`: dataframe que va de segundo.
- `on`: es la columna o la lista de columnas que determinan qué filas de una tabla coinciden con qué filas de la segunda tabla. Comúnmente a estas variables se les llaman las llaves del pegue y debe identificar a cada observación de forma única. A veces, las columnas que desea fusionar tienen nombres diferentes en los datos. Por ejemplo, suponga que tiene dos bases de datos, una que registra el dinero mensual gastado por persona en almacenes Éxito y otra que tiene características personales de las personas. Usted podría tratar de juntar ambas bases con el identificador de fila o persona de cada base que en este caso podría ser la cédula, sin embargo, en un dataframe tal vez la variable se llame "cc" mientras que en el otro puede que se llame "cédula". En esos casos, puede especificar los nombres de columna por separado para cada marco de datos utilizando los argumentos "left_on" y "right_on".
- `how`: es el método a usar, por defecto Pandas usa el método "inner". Más adelante exploraremos más al respecto.

<center>
<div>
<img src="img/merges.png" width="400"/>
</div>
</center>

Tenemos cuatro grandes métodos para relacionar las bases porque no siempre tenemos una coincidencia uno a uno (one to one) entre las filas. Estos cuatro métodos afectan la forma en que Pandas trata los datos no coincidentes y eso es lo que veremos más adelante.

<center>
<div>
<img src="img/one-many.png" width="400"/>
</div>
</center>



In [460]:
import pandas as pd
import numpy as np

# ejemplos de pegues
left_dataframe = pd.DataFrame({"ID": [1,2,3,4], "left_side": "Izquierda"})
right_dataframe = pd.DataFrame({"ID": [3,4,5,6], "right_side": "Derecha"})

In [461]:
left_dataframe

,ID,left_side
0,1,Izquierda
1,2,Izquierda
2,3,Izquierda
3,4,Izquierda


In [462]:
right_dataframe

,ID,right_side
0,3,Derecha
1,4,Derecha
2,5,Derecha
3,6,Derecha


#### Left merge
En un Left merge lo que más nos interesa son los datos del lado IZQUIERDO a los cuales queremos pegarles columnas de una base de datos en el lado DERECHO.

Para hacer eso, cortamos las filas en el marco de datos DERECHO y pegamos partes en el marco de datos IZQUIERDO. Recuerde, nos preocupamos principalmente por el lado IZQUIERDO y solo queremos datos del lado DERECHO si tiene alguna de las mismas ID. Entonces, si algo en el marco de datos DERECHO no coincide o no existe, entonces tenemos que hacer cosas para mantener las columnas de la misma longitud. Lo hacemos agregando NaN para llenar el vacío o descartando algunas filas por completo.

En este ejemplo, el lado IZQUIERDO tiene los ID 1, 2, 3 y 4:
- El lado DERECHO no tiene ID 1 o 2, por lo que agregamos NaN porque necesitamos que las columnas tengan la misma longitud.
- El lado DERECHO tiene datos para los ID 3 y 4, así que lo agregamos como una nueva columna.
- El lado IZQUIERDO no tiene ID 5 o 6, por lo que no necesitamos esa información del DERECHO y se descarta.

<center>
<div>
<img src="img/left_merge.png" width="400"/>
</div>
</center>

In [463]:
# Left merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "left")

,ID,left_side,right_side
0,1,Izquierda,NaN
1,2,Izquierda,NaN
2,3,Izquierda,Derecha
3,4,Izquierda,Derecha


#### Right merge
Los Right merges funcionan igual que los Left merges, la diferencia es que nos preocupamos principalmente por el lado DERECHO y nos gustaría agregar datos desde el IZQUIERDO si tienen ID coincidentes.

<center>
<div>
<img src="img/right_merge.png" width="400"/>
</div>
</center>

In [464]:
# Right merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "right")

,ID,left_side,right_side
0,3,Izquierda,Derecha
1,4,Izquierda,Derecha
2,5,NaN,Derecha
3,6,NaN,Derecha


#### Inner merge
Con un Inner merge, cortamos ambos marcos de datos y solo pegamos las cosas que coinciden. Si una ID no está en ambos marcos de datos, no la mantenemos y no agregamos NaN.

<center>
<img src="img/right_merge.png" width="400"/>
</center>

In [465]:
# Inner merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "inner")

,ID,left_side,right_side
0,3,Izquierda,Derecha
1,4,Izquierda,Derecha


#### Outer merge
Con un Outer merge, cortamos ambos marcos de datos y mantenemos todo de ambos lados. Luego agregamos NaN para llenar los espacios en blanco.

<center>
<img src="img/outer_merge.png" width="400"/>
</center>

In [466]:
# Outer merge con "ID" como llave
pd.merge(left = left_dataframe, right = right_dataframe, on = "ID", how = "outer")

,ID,left_side,right_side
0,1,Izquierda,NaN
1,2,Izquierda,NaN
2,3,Izquierda,Derecha
3,4,Izquierda,Derecha
4,5,NaN,Derecha
5,6,NaN,Derecha


Ahora haremos un ejemplo más práctico. Importaremos dos bases de datos sobre usuarios que califican restaurantes en internet. La primera llamada `payment` informa el método de pago favorito para cada cliente y la segunda, `profile` describe algunas características sociodemográficas de los clientes. En este caso nuestra idea es juntar ambas bases de datos.

In [467]:
# Importe bases de datos
payment = pd.read_csv("../Datos/userpayment.csv")
profile = pd.read_csv("../Datos/userprofile.csv")

In [468]:
# Analicemos la estructura de las bases
payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    177 non-null    object
 1   Upayment  177 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [469]:
payment.head()

,userID,Upayment
0,U1001,cash
1,U1002,cash
2,U1003,cash
3,U1004,cash
4,U1004,bank_debit_cards


In [470]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   userID            138 non-null    object 
 1   latitude          138 non-null    float64
 2   longitude         138 non-null    float64
 3   smoker            138 non-null    object 
 4   drink_level       138 non-null    object 
 5   dress_preference  138 non-null    object 
 6   ambience          138 non-null    object 
 7   transport         138 non-null    object 
 8   marital_status    138 non-null    object 
 9   hijos             138 non-null    object 
 10  birth_year        138 non-null    int64  
 11  interest          138 non-null    object 
 12  personality       138 non-null    object 
 13  religion          138 non-null    object 
 14  activity          138 non-null    object 
 15  color             138 non-null    object 
 16  weight            138 non-null    int64  
 1

In [471]:
profile.head()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69


Note que ambas bases tienen diferente número de observaciones, `profile` tiene 133 observaciones mientras que `payment` tiene 177 clientes. Esto quiere decir que, aunque profile es una caracterización más completa de los clientes, payment tiene más observaciones. Adicionalmente, ninguna de las bases tiene NAs.

Al parecer `userID` corresponde a la llave/identificador de cada cliente/fila. Revisemos que no hayan duplicados!

In [472]:
payment.userID.duplicated().sum()

44

In [473]:
profile.userID.duplicated().sum()

0

Mientras que `profile` no tiene duplicados, `payment` tiene 44 duplicados, vamos a revisarlos. Al parecer ambas bases tienen los mismos clientes, lo que pasa es que algunos tienen más de un tipo de método de pago.

In [474]:
# Devolver todos los duplicados
payment.loc[payment.userID.duplicated(False),]

,userID,Upayment
3,U1004,cash
4,U1004,bank_debit_cards
12,U1012,cash
13,U1012,bank_debit_cards
14,U1013,MasterCard-Eurocard
...,...,...
155,U1117,cash
159,U1121,cash
160,U1121,bank_debit_cards
170,U1133,bank_debit_cards


In [475]:
# Vamos a eliminar los duplicados dejando solo la primera observación. Suponemos que el primer método de pago es el 
# más deseado
payment = payment.drop_duplicates(subset = ["userID"], keep = "first").reset_index(drop = True)

Como la base que más nos interesa es la de `profile` vamos a hacer que esta sea nuestra base de la IZQUIERDA y hacer un LEFT merge

In [476]:
df = pd.merge(left = profile, right = payment, on = "userID", how = "left")
df.head()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height,Upayment
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77,cash
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87,cash
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69,cash
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53,cash
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69,cash


Debemos revisar que todos los elementos en profile hayan encontrado un match exacto en payment

In [477]:
df["Upayment"].isna().sum()

5

Upa! Tenemos 5 NAs. Eso quiere decir que hay 5 usuarios/clientes en profile que no están en payment! Revisemos

In [478]:
usuarios_faltantes = df.loc[df["Upayment"].isna(), "userID"].values
usuarios_faltantes

array(['U1024', 'U1025', 'U1088', 'U1122', 'U1130'], dtype=object)

In [479]:
payment["userID"].isin(usuarios_faltantes).sum()

0

In [480]:
# En efecto, estos 5 usuarios no están en la base de payment
set(profile["userID"]) - set(payment["userID"])

{'U1024', 'U1025', 'U1088', 'U1122', 'U1130'}

In [481]:
# Sin embargo, en la base de profile sí están todos los usuarios de payment
set(payment["userID"]) - set(profile["userID"])

set()

In [482]:
profile.shape

(138, 19)

In [483]:
payment.shape

(133, 2)

#### Ejercicios

1. Importe los dos csvs de encuestas: `encuesta_hogares1.csv` y `encuesta_hogares2.csv`

In [484]:
import os

In [485]:
os.getcwd()

'c:\\Users\\Lucas\\OneDrive - Universidad de los Andes\\Complementarias\\Proyectos de analítica en Python\\Introducción al análisis de datos en Python\\Clase 5\\Scripts'

In [486]:
encuesta1 = pd.read_csv('..\Datos\encuesta_hogares1.csv') 
encuesta2 = pd.read_csv('..\Datos\encuesta_hogares2.csv') 

2. Inspeccione sus datos

In [487]:
encuesta1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   DIRECTORIO      250 non-null    int64
 1   HOGAR_NUMERO    250 non-null    int64
 2   PERSONA_NUMERO  250 non-null    int64
 3   edad            250 non-null    int64
 4   sexo            250 non-null    int64
dtypes: int64(5)
memory usage: 9.9 KB


In [488]:
encuesta1.head()

,DIRECTORIO,HOGAR_NUMERO,PERSONA_NUMERO,edad,sexo
0,36228,1,2,53,2
1,36252,1,2,64,2
2,36281,1,2,88,2
3,36297,1,2,25,2
4,36313,1,1,66,2


In [489]:
encuesta2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   DIRECTORIO      250 non-null    int64
 1   HOGAR_NUMERO    250 non-null    int64
 2   PERSONA_NUMERO  250 non-null    int64
 3   edad            250 non-null    int64
 4   sexo            250 non-null    int64
dtypes: int64(5)
memory usage: 9.9 KB


In [490]:
encuesta2.head()

,DIRECTORIO,HOGAR_NUMERO,PERSONA_NUMERO,edad,sexo
0,43678,1,1,40,2
1,43709,1,4,24,2
2,43721,1,1,42,1
3,43764,1,1,36,1
4,43908,1,1,48,1


3. ¿Qué tipo de pegue podría hacer?

**R// Concatenar por filas**

4. Realice el pegue que definió anteriormente

In [491]:
# Pegar filas
encuesta = pd.concat([encuesta1, encuesta2], axis = 0)
encuesta.head()

,DIRECTORIO,HOGAR_NUMERO,PERSONA_NUMERO,edad,sexo
0,36228,1,2,53,2
1,36252,1,2,64,2
2,36281,1,2,88,2
3,36297,1,2,25,2
4,36313,1,1,66,2


5. Importe la hoja parteA de `info_accidentes.xlsx`

In [492]:
accidentes1 = pd.read_excel('../Datos/info_accidentes.xlsx', sheet_name = 'parteA') 

6. Importe la hoja parteB de `info_accidentes.xlsx`

In [493]:
accidentes2 = pd.read_excel('../Datos/info_accidentes.xlsx', sheet_name = 'parteB') 

7. Inspecciones ambas bases

In [494]:
accidentes1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34931 entries, 0 to 34930
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fecha             34931 non-null  object 
 1   GravedadNombre    34931 non-null  object 
 2   ClaseNombre       34931 non-null  object 
 3   ChoqueNombre      34931 non-null  object 
 4   ObjetoFijoCodigo  34931 non-null  object 
 5   ObjetoFijoNombre  34931 non-null  object 
 6   OtraClase         34931 non-null  object 
 7   NombreOtraClase   34931 non-null  object 
 8   Latitud           34931 non-null  float64
 9   Longitud          34931 non-null  float64
 10  Direccion         34931 non-null  object 
dtypes: float64(2), object(9)
memory usage: 2.9+ MB


In [495]:
accidentes1.head()

,Fecha,GravedadNombre,ClaseNombre,ChoqueNombre,ObjetoFijoCodigo,ObjetoFijoNombre,OtraClase,NombreOtraClase,Latitud,Longitud,Direccion
0,01/13/2016 12:00:00 AM,Con Heridos,Choque,Vehiculo,,,,,0.0,0.0,KR 101-CL 83 02
1,2016-12-01 00:00:00,Con Heridos,Atropello,,,,,,0.0,0.0,CL 147-KR 7 02
2,01/13/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,CL 19-KR 5 SE 02
3,01/17/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,AC 80-KR 107 02
4,01/18/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,TR 14-CL 48 S 05


In [496]:
accidentes2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34931 entries, 0 to 34930
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Fecha           34931 non-null  object 
 1   Direccion       34931 non-null  object 
 2   TipoVia1        34931 non-null  object 
 3   NumeroVia1      34931 non-null  object 
 4   LetraVia1       34931 non-null  object 
 5   CardinalVia1    34931 non-null  object 
 6   TipoVia2        34931 non-null  object 
 7   NumeroVia2      34625 non-null  float64
 8   LetraVia2       34931 non-null  object 
 9   CardinalVia2    34931 non-null  object 
 10  Localidad       34931 non-null  object 
 11  HoraOcurrencia  34931 non-null  object 
 12  TipoDiseño      34931 non-null  object 
 13  TipoTiempo      34931 non-null  object 
 14  TotalMuertos    34931 non-null  int64  
 15  TotalHeridos    34931 non-null  int64  
dtypes: float64(1), int64(2), object(13)
memory usage: 4.3+ MB


In [497]:
accidentes2.head()

,Fecha,Direccion,TipoVia1,NumeroVia1,LetraVia1,CardinalVia1,TipoVia2,NumeroVia2,LetraVia2,CardinalVia2,Localidad,HoraOcurrencia,TipoDiseño,TipoTiempo,TotalMuertos,TotalHeridos
0,01/13/2016 12:00:00 AM,KR 101-CL 83 02,KR,101,,,CL,83.0,,,ENGATIVA,12/31/1899 10:30:00 PM,Interseccion,Normal,0,2
1,2016-12-01 00:00:00,CL 147-KR 7 02,CL,147,,,KR,7.0,,,USAQUEN,12/31/1899 03:40:00 PM,Interseccion,Normal,0,1
2,01/13/2016 12:00:00 AM,CL 19-KR 5 SE 02,CL,19,,S,KR,5.0,,E,SAN CRISTOBAL,12/31/1899 07:20:00 AM,Tramo de Via,Normal,0,0
3,01/17/2016 12:00:00 AM,AC 80-KR 107 02,AC,80,,,KR,107.0,,,ENGATIVA,12/31/1899 06:10:00 AM,Tramo de Via,Normal,0,0
4,01/18/2016 12:00:00 AM,TR 14-CL 48 S 05,TR,14,,,CL,48.0,,S,RAFAEL URIBE URIBE,12/31/1899 05:00:00 PM,Tramo de Via,Normal,0,0


8. Defina las variables que serán las llaves de su pegue y luego unifiqueles el formato/tipo.

In [498]:
accidentes1['Direccion'] = accidentes1['Direccion'].astype(str) 
accidentes1['Fecha'] = accidentes1['Fecha'].astype(str)
accidentes2['Direccion'] = accidentes2['Direccion'].astype(str)
accidentes2['Fecha'] = accidentes2['Fecha'].astype(str)

9. Realice un *inner_join*

In [499]:
# Hace un inner join
accidentes = pd.merge(accidentes1, accidentes2, on = ['Direccion', 'Fecha'])

In [500]:
accidentes.head()

,Fecha,GravedadNombre,ClaseNombre,ChoqueNombre,ObjetoFijoCodigo,ObjetoFijoNombre,OtraClase,NombreOtraClase,Latitud,Longitud,...,TipoVia2,NumeroVia2,LetraVia2,CardinalVia2,Localidad,HoraOcurrencia,TipoDiseño,TipoTiempo,TotalMuertos,TotalHeridos
0,01/13/2016 12:00:00 AM,Con Heridos,Choque,Vehiculo,,,,,0.0,0.0,...,CL,83.0,,,ENGATIVA,12/31/1899 10:30:00 PM,Interseccion,Normal,0,2
1,2016-12-01 00:00:00,Con Heridos,Atropello,,,,,,0.0,0.0,...,KR,7.0,,,USAQUEN,12/31/1899 03:40:00 PM,Interseccion,Normal,0,1
2,01/13/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,KR,5.0,,E,SAN CRISTOBAL,12/31/1899 07:20:00 AM,Tramo de Via,Normal,0,0
3,01/17/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,KR,107.0,,,ENGATIVA,12/31/1899 06:10:00 AM,Tramo de Via,Normal,0,0
4,01/18/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,CL,48.0,,S,RAFAEL URIBE URIBE,12/31/1899 05:00:00 PM,Tramo de Via,Normal,0,0


### Exportar bases de datos
En Python, Pandas ofrece varias opciones para exportar los datos a diferentes formatos de archivo. Aquí hay algunos ejemplos de cómo exportar datos de Pandas:
1. Exportar a un archivo CSV con [to_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html)

In [501]:
# Creamos un dataframe para el ejemplo
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
df

,col1,col2
0,1,3
1,2,4


In [502]:
df.to_csv('../Datos/df_coma.csv', sep = ';')

In [503]:
df.to_csv('../Datos/df_punto_coma.csv', sep = ';')

2. Exportar a un archivo Excel con [to_excel()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html)

In [504]:
df.to_excel('../Datos/df.xlsx', sheet_name = 'Prueba', index = False)

3. Exportar a un archivo Pickle con [to_pickle](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html)

En general, los archivos de pickle son más compactos que los archivos de Excel o CSV, especialmente para conjuntos de datos grandes y complejos. Esto se debe a que los archivos de pickle utilizan un formato binario, que es más eficiente en cuanto al espacio de almacenamiento que los formatos de texto plano como CSV y Excel.

Sin embargo, hay algunas cosas a tener en cuenta:

- Los archivos de pickle no son fácilmente legibles por humanos, mientras que los archivos CSV y Excel sí lo son. Por lo tanto, si necesita compartir datos con personas que no son programadores, es posible que desee utilizar un formato más legible.

- Los archivos de pickle pueden ser más lentos de procesar que los archivos CSV y Excel, especialmente para conjuntos de datos pequeños. Esto se debe a que la lectura y escritura de archivos de pickle implica la serialización y deserialización de objetos, lo que puede ser más lento que la lectura y escritura de archivos de texto plano.

En resumen, si está trabajando con conjuntos de datos grandes y complejos y no necesita compartir los datos con personas que no son programadores, los archivos de pickle pueden ser una opción más eficiente en cuanto al espacio de almacenamiento. Sin embargo, si necesita compartir los datos con otros o necesita procesar conjuntos de datos pequeños, es posible que prefiera utilizar un formato más legible y fácil de procesar, como CSV o Excel.

**Es importante tener en cuenta que el módulo pickle no es seguro contra ataques maliciosos, ya que puede ejecutar código arbitrario al deserializar objetos. Por lo tanto, solo se deben deserializar objetos de fuentes confiables.**

In [505]:
df.to_pickle('../Datos/df.gzip', compression = 'gzip')

In [506]:
pd.read_pickle('../Datos/df.gzip', compression = "gzip")

,col1,col2
0,1,3
1,2,4


In [507]:
for i in os.listdir("../Datos/"):
    if "df" in i:
        print("El archivo", i, "pesa", os.stat("../Datos/" + i).st_size, "bytes")

El archivo df.gzip pesa 479 bytes
El archivo df.xlsx pesa 5428 bytes
El archivo df_coma.csv pesa 26 bytes
El archivo df_punto_coma.csv pesa 26 bytes


In [508]:
accidentes.to_pickle('../Datos/info_accidentes.gzip', compression = 'gzip')

In [509]:
for i in os.listdir("../Datos/"):
    if "info_accidentes" in i:
        print("El archivo", i, "pesa", np.round(os.stat("../Datos/" + i).st_size/ (1024 * 1024), 0), "megabytes")

El archivo info_accidentes.csv pesa 5.0 megabytes
El archivo info_accidentes.gzip pesa 1.0 megabytes
El archivo info_accidentes.xlsx pesa 4.0 megabytes


### Groupby


Uno de los métodos más útiles para los analistas de datos es `.groupby()`. Este método permite dividir los datos en grupos y a cada uno de estos aplicarles una función de agregación.

Veamos el siguiente ejemplo para entender este concepto mejor:

In [510]:
df = pd.read_excel("../Datos/ejemplo_groupby.xlsx")
df

,animal,age,weight,length
0,hamster,1,7,8
1,alligator,9,13,6
2,hamster,4,8,9
3,cat,13,12,1
4,snake,14,11,8
5,cat,10,8,9
6,hamster,2,10,5
7,cat,4,14,6
8,cat,14,9,6
9,snake,7,11,6


Note que tenemos un `dataframe` con cuatro tipos de animales: 
- alligators (cocodrilos 🐊)
- cats (gatos 🐱)
- snakes (serpientes 🐍)
- hamsters (hamsters 🐹)

Cada una de las filas indican un chequeo en el veterinario donde se registra edad, peso y largo del animal. Por ende, usted como investigador quiere estudiar algunas estadísticas descriptivas por especie. Por ejemplo ¿Cuál es el peso promedio de cada especie?

In [511]:
# El primer paso es agrupar por animal
animal_groups = df.groupby("animal")

In [512]:
animal_groups

In [513]:
# Veamos la conformación de cada uno de los grupos. ¿En qué filas aparece cada animal?
animal_groups.groups

{'alligator': [1, 13], 'cat': [3, 5, 7, 8, 12], 'hamster': [0, 2, 6, 10, 11], 'snake': [4, 9]}

In [514]:
# El segundo paso es aplicar una funcion agregadora
# ¿Cuál es la media del peso por especie?
animal_groups["weight"].mean()

animal
alligator    13.5
cat          10.4
hamster       9.0
snake        11.0
Name: weight, dtype: float64

Visualmente, lo que sucedió fue lo siguiente:

1. Se agrupa los valores únicos de la columna animal.
<center>
<img src = "img/groupby1.jpg" width = "400">
</center>

2. La segmentación de cada grupo se vería de la siguiente manera
<center>
<img src = "img/groupby2.jpg" width = "400">
</center>

3. Se le asignan las otras variables/columnas a cada grupo
<center>
<img src = "img/groupby3.jpg" width = "400">
</center>

4. Se aplica la función agregadora `.mean()` sobre la columna `weight` de cada grupo.
<center>
<img src = "img/groupby4.jpg" width = "400">
</center>


In [515]:
# Probemos otros ejemplos
# ¿Cuál es la edad mediana por animal?
df.groupby("animal")["age"].median()

animal
alligator     8.0
cat          10.0
hamster       2.0
snake        10.5
Name: age, dtype: float64

In [516]:
# ¿Cuál es el largo máximo por animal?
df.groupby("animal")["length"].max()

animal
alligator    6
cat          9
hamster      9
snake        8
Name: length, dtype: int64

In [517]:
# ¿Cuál es la desviación estándar del peso por animal?
df.groupby("animal")["weight"].std()

animal
alligator    0.707107
cat          2.509980
hamster      1.414214
snake        0.000000
Name: weight, dtype: float64

Para seguir practicando, vamos a utilizar los dos métodos vistos en clase (`.merge()` y `.groupby()`) en un solo ejercicio. Vamos a combinar una base de datos que contiene restaurantes y sus respectivas calificaciones con otra que contiene la información del tipo de parqueadero que tiene cada restaurante: `[None, Public, Valet, Yes]`.

La pregunta que queremos resolver con este ejercicio es cómo el parqueadero puede influir en la calificación o percepción de los clientes sobre un restaurante.

In [518]:
# 1. Importe los datos
ratings = pd.read_csv("../Datos/rating_final.csv")
parking = pd.read_csv("../Datos/chefmozparking.csv")

In [519]:
# Inspeccione los datos
ratings.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [520]:
# Se puede ver que userID se refiere al identificador de usuario que calificó al restaurante placeID. 
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userID          1161 non-null   object
 1   placeID         1161 non-null   int64 
 2   rating          1161 non-null   int64 
 3   food_rating     1161 non-null   int64 
 4   service_rating  1161 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 45.5+ KB


In [521]:
# Inspeccionemos la base de parking
parking.head()

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none


In [522]:
# Para cada restaurante (placeID) se tiene una descripción del tipo de parqueadero.
# Estudiemos cuántos tipos de parqueaderos tiene cada restaurante
parking.placeID.value_counts().describe()

count    675.00000
mean       1.04000
std        0.20353
min        1.00000
25%        1.00000
50%        1.00000
75%        1.00000
max        3.00000
Name: placeID, dtype: float64

In [523]:
# Veamos la proporción de tipos de parqueaderos
parking.parking_lot.value_counts(normalize = True)

none                 0.495726
yes                  0.247863
public               0.145299
street               0.045584
fee                  0.031339
valet parking        0.029915
validated parking    0.004274
Name: parking_lot, dtype: float64

In [524]:
# En general cada restaurante tiene un sólo tipo de parqueadero pero hay algunos que tienen más de 1 tipo
# Pregunta: ¿Cuál es la variable con la que queremos hacer el pegue?
# ¿Qué tipo de pegue queremos hacer?

In [525]:
# Queremos hacer el pegue con la variable placeID.
# Debemos verificar que ambas variables estén en el mismo formato
ratings.placeID.dtype

dtype('int64')

In [526]:
parking.placeID.dtype

dtype('int64')

In [527]:
# Esto debe ser True siempre
ratings.placeID.dtype == parking.placeID.dtype

True

In [528]:
# Vamos a hacer un left join porque queremos tener absolutamente todas las calificaciones de los restaurantes
ratings = ratings.merge(parking, on = "placeID", how = "left")

In [529]:
# Veamos que tan bueno estuvo el pegue

# ¿Cuál es la cantidad de NAs o valores faltantes por variable?
ratings.isna().sum()

userID            0
placeID           0
rating            0
food_rating       0
service_rating    0
parking_lot       0
dtype: int64

In [530]:
# ¿Cuál es la proporción de NAs o valores faltantes por variable?
ratings.isna().mean()

userID            0.0
placeID           0.0
rating            0.0
food_rating       0.0
service_rating    0.0
parking_lot       0.0
dtype: float64

In [531]:
# Note que hay 0 NAs en parking_lot sin embargo hay algunos parqueaderos con None. Note que un NA no es lo mismo a None
np.nan == None

False

**¿Como hacemos para analizar las variables de rating a la luz del tipo de parqueo?**

In [532]:
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].mean() \
    .round(2).sort_values("service_rating", ascending = False)

,rating,food_rating,service_rating
parking_lot,,,
valet parking,1.34,1.34,1.34
none,1.20,1.21,1.10
yes,1.21,1.21,1.09
public,1.15,1.22,1.02


¿Qué pasaría si no quisiera tener solo la media sino otras estadísticas más completas?

#### Método .agg()
El método .agg() se puede utilizar después de aplicar un método .groupby() en pandas para realizar operaciones de agregación en los datos de cada grupo.

La sintaxis general de la función .groupby() es la siguiente:
```python
dataframe.groupby(columnas).agg(funciones)
```
Donde:
- dataframe: el DataFrame al que se aplicará la función `groupby()`.
- columnas: la(s) columna(s) que se utilizarán para agrupar los datos.
- funciones: la(s) operación(es) de agregación que se aplicarán a los datos agrupados.

Por ejemplo, para calcular la media, el máximo y el mínimo de las columnas de rating del DataFrame agrupado por la columna 'parking_lot', se puede utilizar la siguiente sintaxis:

In [533]:
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].agg(["min", "mean", "max"])

rating               food_rating               service_rating  \
                 min      mean max         min      mean max            min   
parking_lot                                                                   
none               0  1.203209   2           0  1.212121   2              0   
public             0  1.148352   2           0  1.219780   2              0   
valet parking      0  1.344828   2           0  1.344828   2              0   
yes                0  1.208226   2           0  1.208226   2              0   

                             
                   mean max  
parking_lot                  
none           1.098039   2  
public         1.021978   2  
valet parking  1.344828   2  
yes            1.092545   2

También es posible utilizar varias columnas para agrupar los datos y aplicar diferentes operaciones de agregación a diferentes columnas. Por ejemplo:

In [534]:
ratings.groupby("parking_lot").agg({'rating': ['mean', 'max'], 'food_rating': 'std', 
                                     "service_rating": lambda x: np.percentile(x, 50)})

rating     food_rating service_rating
                   mean max         std       <lambda>
parking_lot                                           
none           1.203209   2    0.783488            1.0
public         1.148352   2    0.804713            1.0
valet parking  1.344828   2    0.813979            2.0
yes            1.208226   2    0.799700            1.0

In [535]:
# Otra sintaxis, en vez de un diccionario, usar tuplas
ratings.groupby("parking_lot").agg(rating_media = ("rating", 'mean'), 
                                   rating_maximo = ("rating", 'max'),
                                   service_rating_mediana = ("service_rating", lambda x: np.percentile(x, 50)))

,rating_media,rating_maximo,service_rating_mediana
parking_lot,,,
none,1.203209,2,1.0
public,1.148352,2,1.0
valet parking,1.344828,2,2.0
yes,1.208226,2,1.0


In [536]:
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].describe()

rating                                              food_rating  \
               count      mean       std  min  25%  50%  75%  max       count   
parking_lot                                                                     
none           561.0  1.203209  0.777857  0.0  1.0  1.0  2.0  2.0       561.0   
public         182.0  1.148352  0.768850  0.0  1.0  1.0  2.0  2.0       182.0   
valet parking   29.0  1.344828  0.768852  0.0  1.0  2.0  2.0  2.0        29.0   
yes            389.0  1.208226  0.770148  0.0  1.0  1.0  2.0  2.0       389.0   

                         ...           service_rating                      \
                   mean  ...  75%  max          count      mean       std   
parking_lot              ...                                                
none           1.212121  ...  2.0  2.0          561.0  1.098039  0.799115   
public         1.219780  ...  2.0  2.0          182.0  1.021978  0.764951   
valet parking  1.344828  ...  2.0  2.0           29.0  1.344828  0.813979   
yes            1.208226  ...  2.0  2.0          389.0  1.092545  0.787578   

                                        
               min  25%  50%  75%  max  
parking_lot                             
none           0.0  0.0  1.0  2.0  2.0  
public         0.0  0.0  1.0  2.0  2.0  
valet parking  0.0  1.0  2.0  2.0  2.0  
yes            0.0  0.0  1.0  2.0  2.0  

[4 rows x 24 columns]

In [537]:
pd.set_option('display.max_columns', None)
ratings.groupby("parking_lot")[["rating", "food_rating", "service_rating"]].describe()

rating                                              food_rating  \
               count      mean       std  min  25%  50%  75%  max       count   
parking_lot                                                                     
none           561.0  1.203209  0.777857  0.0  1.0  1.0  2.0  2.0       561.0   
public         182.0  1.148352  0.768850  0.0  1.0  1.0  2.0  2.0       182.0   
valet parking   29.0  1.344828  0.768852  0.0  1.0  2.0  2.0  2.0        29.0   
yes            389.0  1.208226  0.770148  0.0  1.0  1.0  2.0  2.0       389.0   

                                                           service_rating  \
                   mean       std  min  25%  50%  75%  max          count   
parking_lot                                                                 
none           1.212121  0.783488  0.0  1.0  1.0  2.0  2.0          561.0   
public         1.219780  0.804713  0.0  1.0  1.0  2.0  2.0          182.0   
valet parking  1.344828  0.813979  0.0  1.0  2.0  2.0  2.0           29.0   
yes            1.208226  0.799700  0.0  1.0  1.0  2.0  2.0          389.0   

                                                            
                   mean       std  min  25%  50%  75%  max  
parking_lot                                                 
none           1.098039  0.799115  0.0  0.0  1.0  2.0  2.0  
public         1.021978  0.764951  0.0  0.0  1.0  2.0  2.0  
valet parking  1.344828  0.813979  0.0  1.0  2.0  2.0  2.0  
yes            1.092545  0.787578  0.0  0.0  1.0  2.0  2.0

In [538]:
pd.reset_option('display.max_columns')

In [539]:
pd.set_option('display.max_rows', 50)
ratings[0:50]

,userID,placeID,rating,food_rating,service_rating,parking_lot
0,U1077,135085,2,2,2,public
1,U1077,135038,2,2,1,none
2,U1077,132825,2,2,2,none
3,U1077,135060,1,2,2,none
4,U1068,135104,1,1,2,yes
5,U1068,132740,0,0,0,none
6,U1068,132663,1,1,1,none
7,U1068,132732,0,0,0,none
8,U1068,132630,1,1,1,none
9,U1067,132584,2,2,2,yes


In [540]:
pd.reset_option('display.max_rows')

#### Ejercicios

1. Importe las dos hojas del excel de accidentes y realice un inner join.

In [541]:
# Importar
accidentes1 = pd.read_excel('../Datos/info_accidentes.xlsx', sheet_name = 'parteA') 
accidentes2 = pd.read_excel('../Datos/info_accidentes.xlsx', sheet_name = 'parteB') 

# Unificar formatos
accidentes1['Direccion'] = accidentes1['Direccion'].astype(str) 
accidentes1['Fecha'] = accidentes1['Fecha'].astype(str)
accidentes2['Direccion'] = accidentes2['Direccion'].astype(str)
accidentes2['Fecha'] = accidentes2['Fecha'].astype(str)

# Hace un inner join
accidentes = pd.merge(accidentes1, accidentes2, on = ['Direccion', 'Fecha'])

1.5. Explore la base. Entienda que variables tiene

In [542]:
accidentes.head()

,Fecha,GravedadNombre,ClaseNombre,ChoqueNombre,ObjetoFijoCodigo,ObjetoFijoNombre,OtraClase,NombreOtraClase,Latitud,Longitud,...,TipoVia2,NumeroVia2,LetraVia2,CardinalVia2,Localidad,HoraOcurrencia,TipoDiseño,TipoTiempo,TotalMuertos,TotalHeridos
0,01/13/2016 12:00:00 AM,Con Heridos,Choque,Vehiculo,,,,,0.0,0.0,...,CL,83.0,,,ENGATIVA,12/31/1899 10:30:00 PM,Interseccion,Normal,0,2
1,2016-12-01 00:00:00,Con Heridos,Atropello,,,,,,0.0,0.0,...,KR,7.0,,,USAQUEN,12/31/1899 03:40:00 PM,Interseccion,Normal,0,1
2,01/13/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,KR,5.0,,E,SAN CRISTOBAL,12/31/1899 07:20:00 AM,Tramo de Via,Normal,0,0
3,01/17/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,KR,107.0,,,ENGATIVA,12/31/1899 06:10:00 AM,Tramo de Via,Normal,0,0
4,01/18/2016 12:00:00 AM,Solo Daños,Choque,Vehiculo,,,,,0.0,0.0,...,CL,48.0,,S,RAFAEL URIBE URIBE,12/31/1899 05:00:00 PM,Tramo de Via,Normal,0,0


In [543]:
accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35423 entries, 0 to 35422
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fecha             35423 non-null  object 
 1   GravedadNombre    35423 non-null  object 
 2   ClaseNombre       35423 non-null  object 
 3   ChoqueNombre      35423 non-null  object 
 4   ObjetoFijoCodigo  35423 non-null  object 
 5   ObjetoFijoNombre  35423 non-null  object 
 6   OtraClase         35423 non-null  object 
 7   NombreOtraClase   35423 non-null  object 
 8   Latitud           35423 non-null  float64
 9   Longitud          35423 non-null  float64
 10  Direccion         35423 non-null  object 
 11  TipoVia1          35423 non-null  object 
 12  NumeroVia1        35423 non-null  object 
 13  LetraVia1         35423 non-null  object 
 14  CardinalVia1      35423 non-null  object 
 15  TipoVia2          35423 non-null  object 
 16  NumeroVia2        35103 non-null  float6

In [544]:
for i in accidentes.select_dtypes("object").columns:
    col = accidentes[i]
    if len(col.unique()) < 20:
        print(col.value_counts())
        print("------------------------------------")

Solo Daños     24209
Con Heridos    10654
Con Muertos      560
Name: GravedadNombre, dtype: int64
------------------------------------
Choque               30397
Atropello             3695
Caida de ocupante      879
Volcamiento            353
Otro                    94
Autolesion               4
Incendio                 1
Name: ClaseNombre, dtype: int64
------------------------------------
Vehiculo       29289
                5026
Objeto Fijo     1059
Otro              42
Semoviente         7
Name: ChoqueNombre, dtype: int64
------------------------------------
      34364
10      272
2       269
1       251
6       123
4        42
3        40
5        32
8        19
9         8
7         3
Name: ObjetoFijoCodigo, dtype: int64
------------------------------------
                        34364
Vehiculo estacionado      272
Poste                     264
Muro                      233
Inmueble                  123
Arbol                      40
Barandas                   40
Semaforo        

2. ¿Cuántos muertos y heridos se tiene por Localidad?

In [545]:
accidentes.groupby(['Localidad'])[['TotalHeridos', 'TotalMuertos']].sum() 

,TotalHeridos,TotalMuertos
Localidad,,
ANTONIO NARIÑO,370,19
BARRIOS UNIDOS,630,23
BOSA,856,33
CANDELARIA,65,2
CHAPINERO,635,15
CIUDAD BOLIVAR,890,40
ENGATIVA,1252,65
FONTIBON,843,40
KENNEDY,1964,78


3. ¿Cuántos eventos hubo por Tipo de Diseño?

In [546]:
accidentes.groupby(['TipoDiseño']).size() 

TipoDiseño
Cicloruta           23
Glorieta           413
Interseccion      7538
Lote o predio      314
Paso a nivel        82
Paso elevado       286
Paso inferior      211
Ponton               2
Puente              97
Tramo de Via     26436
Tunel                7
Via peatonal        13
Via troncal          1
dtype: int64

4. Muestre la suma de muertos y heridos por localidad y tipo de diseño 

In [547]:
pd.set_option("display.max_rows", 170)
accidentes.groupby(['Localidad', 'TipoDiseño'])[['TotalHeridos', 'TotalMuertos']].sum() 

TotalHeridos  TotalMuertos
Localidad          TipoDiseño                               
ANTONIO NARIÑO     Glorieta                  2             0
                   Interseccion            132             4
                   Lote o predio             1             0
                   Paso a nivel              0             0
                   Paso elevado              2             0
                   Paso inferior             0             0
                   Puente                    2             0
                   Tramo de Via            231            15
BARRIOS UNIDOS     Cicloruta                 2             0
                   Glorieta                  8             0
                   Interseccion            243             7
                   Lote o predio             0             0
                   Paso a nivel              0             0
                   Paso elevado              1             0
                   Paso inferior             0             0
                   Ponton                    0             0
                   Puente                    1             0
                   Tramo de Via            374            16
                   Via peatonal              1             0
BOSA               Cicloruta                 3             0
                   Glorieta                  4             0
                   Interseccion            217             6
                   Lote o predio             1             0
                   Paso a nivel              1             0
                   Paso elevado              1             0
                   Paso inferior             2             0
                   Puente                    1             0
                   Tramo de Via            623            27
                   Tunel                     3             0
                   Via peatonal              0             0
                   Via troncal               0             0
CANDELARIA         Glorieta                  0             0
                   Interseccion             15             1
                   Lote o predio             0             0
                   Paso inferior             0             0
                   Tramo de Via             50             1
CHAPINERO          Cicloruta                 0             0
                   Glorieta                  1             0
                   Interseccion            164             1
                   Lote o predio             2             0
                   Paso a nivel              2             0
                   Paso elevado              6             0
                   Paso inferior             2             0
                   Ponton                    0             0
                   Puente                    1             0
                   Tramo de Via            456            14
                   Tunel                     1             0
CIUDAD BOLIVAR     Glorieta                  6             0
                   Interseccion            154             3
                   Lote o predio             3             1
                   Paso a nivel              3             0
                   Paso elevado              0             0
                   Paso inferior             7             0
                   Puente                    0             0
                   Tramo de Via            717            35
                   Via peatonal              0             1
ENGATIVA           Cicloruta                 6             0
                   Glorieta                  4             0
                   Interseccion            389             6
                   Lote o predio             3             0
                   Paso a nivel              0             0
                   Paso elevado             10             2
                   Paso inferior             5             0
                   Puente                    7             0
        

In [548]:
pd.reset_option("display.max_rows")

5. Muestre la suma y la desviación estándar de muertos y heridos por localidad y tipo de diseño 

In [549]:
accidentes.groupby(['Localidad', 'TipoDiseño'])[['TotalHeridos', 'TotalMuertos']] \
    .agg(['sum', 'std'])

TotalHeridos           TotalMuertos          
                                      sum       std          sum       std
Localidad      TipoDiseño                                                 
ANTONIO NARIÑO Glorieta                 2  0.323381            0  0.000000
               Interseccion           132  0.951022            4  0.137019
               Lote o predio            1  0.707107            0  0.000000
               Paso a nivel             0  0.000000            0  0.000000
               Paso elevado             2  1.154701            0  0.000000
...                                   ...       ...          ...       ...
USME           Glorieta                 0       NaN            0       NaN
               Interseccion            32  0.656144            0  0.000000
               Lote o predio            5  0.527046            0  0.000000
               Paso inferior            1  0.500000            0  0.000000
               Tramo de Via           324  0.844235           21  0.200205

[170 rows x 4 columns]

### Manipulación de fechas

Note que cuando inspeccionamos nuestra base, las columnas `Fecha` y `HoraOcurrencia` son de tipo `object` o sea texto. Sin embargo, las fechas tienen un tipo particular en `pandas` y para poder operar correctamente estas columnas, debemos formatearlas en tipo `datetime`. 

In [550]:
accidentes[["Fecha", "HoraOcurrencia"]].head()

,Fecha,HoraOcurrencia
0,01/13/2016 12:00:00 AM,12/31/1899 10:30:00 PM
1,2016-12-01 00:00:00,12/31/1899 03:40:00 PM
2,01/13/2016 12:00:00 AM,12/31/1899 07:20:00 AM
3,01/17/2016 12:00:00 AM,12/31/1899 06:10:00 AM
4,01/18/2016 12:00:00 AM,12/31/1899 05:00:00 PM


In [551]:
accidentes[["Fecha", "HoraOcurrencia"]].dtypes

Fecha             object
HoraOcurrencia    object
dtype: object

Note que en la variable `Fecha` la hora por default es 12:00 AM. La hora como tal del evento está consignada en la variable `HoraOcurrencia`. Sin embargo, en esa variable la fecha está mal.

Primero codifiquemos la fecha, es lo más fácil. Usando la función `pd.to_datetime()` tenemos la ventaja de que Python infiere el formato de la fecha que en este caso es %MM/%DD/%YYYY

In [552]:
accidentes["Fecha"] = pd.to_datetime(accidentes["Fecha"])
accidentes["Fecha"]

0       2016-01-13
1       2016-12-01
2       2016-01-13
3       2016-01-17
4       2016-01-18
           ...    
35418   2016-12-31
35419   2016-12-29
35420   2016-12-30
35421   2016-12-30
35422   2016-12-31
Name: Fecha, Length: 35423, dtype: datetime64[ns]

In [553]:
# Podemos extraer elementos relevantes. Número del día
accidentes["Fecha"].dt.day

0        13
1         1
2        13
3        17
4        18
         ..
35418    31
35419    29
35420    30
35421    30
35422    31
Name: Fecha, Length: 35423, dtype: int64

In [554]:
# ¿Qué día de la semana es? 
accidentes["Fecha"].dt.day_of_week

0        2
1        3
2        2
3        6
4        0
        ..
35418    5
35419    3
35420    4
35421    4
35422    5
Name: Fecha, Length: 35423, dtype: int64

In [555]:
# 0 - Lunes
# 1 - Martes
# 2 - Miércoles
# 3 - Jueves
# 4 - Viernes
# 5 - Sábado
# 6 - Domingo
pd.to_datetime("2023-03-30").day_of_week

3

In [556]:
# ¿Qué día del año es?
accidentes["Fecha"].dt.day_of_year

0         13
1        336
2         13
3         17
4         18
        ... 
35418    366
35419    364
35420    365
35421    365
35422    366
Name: Fecha, Length: 35423, dtype: int64

In [557]:
# Primer día del año: 1
pd.to_datetime("2023-01-01").day_of_year

1

In [558]:
# Último día del año no bisiesto: 365
pd.to_datetime("2023-12-31").day_of_year

365

In [559]:
# Último día del año bisiesto: 366
pd.to_datetime("2020-12-31").day_of_year

366

In [560]:
# ¿Cuántos días hay en el mes?
accidentes["Fecha"].dt.days_in_month

0        31
1        31
2        31
3        31
4        31
         ..
35418    31
35419    31
35420    31
35421    31
35422    31
Name: Fecha, Length: 35423, dtype: int64

In [561]:
pd.to_datetime("2023-01-01").days_in_month

31

In [562]:
pd.to_datetime("2023-02-01").days_in_month

28

In [563]:
pd.to_datetime("2023-04-01").days_in_month

30

In [564]:
# Nombre del día
accidentes["Fecha"].dt.day_name()

0        Wednesday
1         Thursday
2        Wednesday
3           Sunday
4           Monday
           ...    
35418     Saturday
35419     Thursday
35420       Friday
35421       Friday
35422     Saturday
Name: Fecha, Length: 35423, dtype: object

In [565]:
# Extraer mes
accidentes["Fecha"].dt.month

0         1
1        12
2         1
3         1
4         1
         ..
35418    12
35419    12
35420    12
35421    12
35422    12
Name: Fecha, Length: 35423, dtype: int64

In [566]:
# Extraer nombre del mes
accidentes["Fecha"].dt.month_name()

0         January
1        December
2         January
3         January
4         January
           ...   
35418    December
35419    December
35420    December
35421    December
35422    December
Name: Fecha, Length: 35423, dtype: object

In [567]:
# Extraer año
accidentes["Fecha"].dt.year

0        2016
1        2016
2        2016
3        2016
4        2016
         ... 
35418    2016
35419    2016
35420    2016
35421    2016
35422    2016
Name: Fecha, Length: 35423, dtype: int64

In [568]:
# Se puede extraer la semana del año
accidentes["Fecha"].dt.isocalendar()

,year,week,day
0,2016,2,3
1,2016,48,4
2,2016,2,3
3,2016,2,7
4,2016,3,1
...,...,...,...
35418,2016,52,6
35419,2016,52,4
35420,2016,52,5
35421,2016,52,5


In [569]:
accidentes["Fecha"].dt.isocalendar().week

0         2
1        48
2         2
3         2
4         3
         ..
35418    52
35419    52
35420    52
35421    52
35422    52
Name: week, Length: 35423, dtype: UInt32

¿Cuántos accidentes hubo por mes?

In [570]:
accidentes.assign(mes = lambda x: x.Fecha.dt.month_name()).groupby(["mes"]).size()

mes
April        2890
August       3145
December     3009
February     2694
January      2587
July         2929
June         2871
March        2727
May          3190
November     3151
October      3159
September    3071
dtype: int64

¿Cuántos accidentes hubo por día de la semana? ¿Cuántos muertos y heridos?

In [571]:
accidentes.assign(dia = lambda x: x.Fecha.dt.day_name()).groupby(["dia"]) \
    [['TotalMuertos', 'TotalHeridos']].agg(["size", "sum"])

TotalMuertos     TotalHeridos      
                  size sum         size   sum
dia                                          
Friday            5303  73         5303  2109
Monday            4927  83         4927  2082
Saturday          5186  93         5186  2039
Sunday            4090  78         4090  1881
Thursday          5387  87         5387  2205
Tuesday           5254  89         5254  2123
Wednesday         5276  77         5276  2120

In [572]:
# Creemos una matriz de ocurrencias
accidentes["dia"] = accidentes.Fecha.dt.day
accidentes["mes"] = accidentes.Fecha.dt.month
pd.crosstab(accidentes.dia, accidentes.mes)

mes,1,2,3,4,5,6,7,8,9,10,11,12
dia,,,,,,,,,,,,
1,28,29,40,56,70,56,51,76,62,40,36,79
2,104,102,119,51,68,99,50,107,89,77,113,106
3,90,99,106,118,105,64,103,97,94,86,113,105
4,122,106,73,117,116,83,109,100,92,64,96,95
5,48,122,117,101,112,114,105,64,57,111,125,102
6,115,104,102,98,64,56,126,123,119,121,113,67
7,106,74,81,72,118,124,92,89,94,62,87,94
8,102,117,95,92,95,119,57,111,119,105,98,106
9,92,112,93,80,95,117,105,118,118,105,75,132


Construya en la base accidentes la columna que indique si es fin de semana o no.

In [573]:
accidentes['fds'] = ((accidentes.dia==5) | (accidentes.dia==6))
accidentes['fds'].value_counts()

False    33037
True      2386
Name: fds, dtype: int64

Muestre el número de accidentes por si fue en fin de semana y Gravedad del accidente

In [574]:
accidentes.groupby(['fds', 'GravedadNombre']).size().reset_index()

,fds,GravedadNombre,0
0,False,Con Heridos,9943
1,False,Con Muertos,513
2,False,Solo Daños,22581
3,True,Con Heridos,711
4,True,Con Muertos,47
5,True,Solo Daños,1628


Ahora vamos a formatear la hora

In [575]:
accidentes["HoraOcurrencia"] = pd.to_datetime(accidentes["HoraOcurrencia"])
accidentes["HoraOcurrencia"]

0       1899-12-31 22:30:00
1       1899-12-31 15:40:00
2       1899-12-31 07:20:00
3       1899-12-31 06:10:00
4       1899-12-31 17:00:00
                ...        
35418   1899-12-31 13:20:00
35419   1899-12-31 17:20:00
35420   1899-12-31 05:10:00
35421   1899-12-31 09:00:00
35422   1899-12-31 12:00:00
Name: HoraOcurrencia, Length: 35423, dtype: datetime64[ns]

Note que la fecha está errada entonces nos vamos a deshacer de ella

In [576]:
# Debemos definir el formato de horas que queremos
accidentes["HoraOcurrencia2"] = accidentes["HoraOcurrencia"].dt.strftime('%H:%M:%S')
accidentes["HoraOcurrencia2"] # Note que la columna se volvió un object

0        22:30:00
1        15:40:00
2        07:20:00
3        06:10:00
4        17:00:00
           ...   
35418    13:20:00
35419    17:20:00
35420    05:10:00
35421    09:00:00
35422    12:00:00
Name: HoraOcurrencia2, Length: 35423, dtype: object

In [577]:
accidentes["HoraOcurrencia"].dt.minute

0        30
1        40
2        20
3        10
4         0
         ..
35418    20
35419    20
35420    10
35421     0
35422     0
Name: HoraOcurrencia, Length: 35423, dtype: int64

In [578]:
accidentes["HoraOcurrencia"].dt.second

0        0
1        0
2        0
3        0
4        0
        ..
35418    0
35419    0
35420    0
35421    0
35422    0
Name: HoraOcurrencia, Length: 35423, dtype: int64

In [579]:
accidentes["HoraOcurrencia2"].dt.hour

AttributeError: Can only use .dt accessor with datetimelike values